# Demo notebook

In [ ]:
import scanpy as sc
import pathlib
import configparser

import data_reading
import data_analysis
import data_plotting

## Setup  
- Read the configuration file (*configuration.txt*)
- Prepare working directories

In [ ]:
# Reading configuration file
config = configparser.ConfigParser()
config.read('configuration.txt')

# Select directories
data_source = pathlib.Path(config.get('paths', 'data_source'))
coordinates_source = pathlib.Path(config.get('paths', 'coordinates_source'))
figures_dir = pathlib.Path(config.get('paths', 'figures_dir'))
temp_dir = pathlib.Path(config.get('paths', 'temp_dir'))

# Scanpy parameters
sc.settings.verbose = 3
sc.set_figure_params(config.items('scanpy_figures'))
sc.settings.figdir = figures_dir

# Prepare workspace
pathlib.Path(figures_dir).mkdir(parents = True, exist_ok = True)
pathlib.Path(temp_dir).mkdir(parents = True, exist_ok = True)

# Data reading  
Read data from *data_source* and *coordinates_source* and store them as two files into *temp_dir*. Such files are named with unique identifier, stored in *filename* and used in the following steps. The files in *temp_dir* are overwritten at the end of this process.

In [ ]:
filename = data_reading.READ(data_source, coordinates_source, temp_dir, config)

# Data analysis  
## PCA  
Perform PCA on the data stored in *temp_dir* identified by *filename*. For this process use the parameters stored in the configuration file previously read. The files in *temp_dir* are overwritten at the end of this process.

In [ ]:
data_analysis.PCA(filename, temp_dir, config)

## Neighbors graph and UMAP  
Construct the neighbors graph and then perform UMAP on the data stored in *temp_dir* identified by *filename*. For this process use the parameters stored in the configuration file previously read. The files in *temp_dir* are overwritten at the end of this process.

In [ ]:
data_analysis.NEIGHBORS(filename, temp_dir, config)
data_analysis.UMAP(filename, temp_dir, config)

## PhenoGraph  
Identify the clusters by performing PhenoGraph on the data stored in *temp_dir* identified by *filename*. For this process use the parameters stored in the configuration file previously read. The files in *temp_dir* are overwritten at the end of this process.

In [ ]:
data_analysis.PHENOGRAPH(filename, temp_dir, config)

# Plotting  
## UMAP representation  
Represent previous UMAP results colored considering PhenoGraph clustering results. The plot is generated considering the parameters stored in the configuration file previously read. A *.png* version of the plot is also stored in the *figures_dir*.

In [ ]:
data_plotting.UMAP(filename, temp_dir, figures_dir, config)

## Clusters representation  
Represent spatial coordinated previous UMAP results colored considering PhenoGraph clustering results. The plot is generated considering the parameters stored in the configuration file previously read. An interactive *.html* version of the plot is also stored in the *figures_dir*.

In [ ]:
data_plotting.CLUSTERS(filename, temp_dir, figures_dir, config)